# HW2 
*Unisse Chua - uc288*

Your task is to compute the number of restaurants per each cuisine using the provided nyc_restaurant_inspections.csv data set. The CSV file is available for download under the Data Sets page on NYU Classes. The cuisine type can be extracted from the "CUISINE DESCRIPTION" column. Note that, a single restaurant may be inspected multiple times. You must use the "CAMIS" column to keep only unique restaurants while computing the number of restaurants per cuisine.

You MUST turn in a Spark notebook, AND a Spark script that works on the NYU Dumbo cluster. For your script, the input and out file path should be specified through the command line. Basically, I should be able to run your script as:

spark-submit YOUR_SCRIPT.py nyc_restaurant_inspections.csv output_folder

and it will produce an output on HDFS under output_folder.

Using "hadoop fs -getmerge" or through the notebook, I should get the expected output similar to the below:

```python
[('American', 6002),
 ('Chinese', 2399),
 ('Cafe', 1629),
 ('Other', 1296),
 ('Pizza', 1186),
 ('Italian', 1016),
 ('Mexican', 877),
 ('Japanese', 859),
 ('Latin (Cuban, Dominican, Puerto Rican, South & Central American)', 840),
 ('Bakery', 733),
 ('Caribbean', 671),
 ('Spanish', 644),
 ('Donuts', 537),
 ('Pizza/Italian', 483),
 ('Chicken', 456),
 ('Sandwiches', 406),
 ('Juice, Smoothies, Fruit Salads', 382),
 ('Hamburgers', 378),
 ('Asian', 371),
 ('Ice Cream, Gelato, Yogurt, Ices', 339),
 ('Indian', 332),
 ('Jewish/Kosher', 327),
 ('French', 319),
 ('Delicatessen', 294),
 ('Thai', 286)]
```

In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
DATA = 'nyc_restaurant_inspections.csv'

In [3]:
yelp = sc.textFile(DATA, use_unicode=True).cache()

list(enumerate(yelp.first().split(',')))

[(0, 'CAMIS'),
 (1, 'DBA'),
 (2, 'BORO'),
 (3, 'BUILDING'),
 (4, 'STREET'),
 (5, 'ZIPCODE'),
 (6, 'PHONE'),
 (7, 'CUISINE DESCRIPTION'),
 (8, 'INSPECTION DATE'),
 (9, 'ACTION'),
 (10, 'VIOLATION CODE'),
 (11, 'VIOLATION DESCRIPTION'),
 (12, 'CRITICAL FLAG'),
 (13, 'SCORE'),
 (14, 'GRADE'),
 (15, 'GRADE DATE'),
 (16, 'RECORD DATE'),
 (17, 'INSPECTION TYPE')]

In [4]:
def extractCuisines(partId, list_of_records):
    if partId==0: 
        next(list_of_records) # skipping the first line
    import csv
    reader = csv.reader(list_of_records)
    for row in reader:
        (cuisine, camis) = (row[7], row[0])
        yield (cuisine, camis)

cuisines = yelp.mapPartitionsWithIndex(extractCuisines)

In [5]:
cuisines.take(5)

[('Chicken', '50015075'),
 ('Jewish/Kosher', '40364529'),
 ('Caribbean', '41580756'),
 ('Italian', '50001255'),
 ('Korean', '50016437')]

In [6]:
print(cuisines.count())

379674


In [7]:
# Use distinct to get the unique restaurant CAMIS per cuisine
cuisineCounts = sorted(cuisines.distinct().countByKey().items(), key=lambda x: -x[1])

In [8]:
cuisineCounts[0:25]

[('American', 6002),
 ('Chinese', 2399),
 ('CafÃ©/Coffee/Tea', 1629),
 ('Other', 1296),
 ('Pizza', 1186),
 ('Italian', 1016),
 ('Mexican', 877),
 ('Japanese', 859),
 ('Latin (Cuban, Dominican, Puerto Rican, South & Central American)', 840),
 ('Bakery', 733),
 ('Caribbean', 671),
 ('Spanish', 644),
 ('Donuts', 537),
 ('Pizza/Italian', 483),
 ('Chicken', 456),
 ('Sandwiches', 406),
 ('Juice, Smoothies, Fruit Salads', 382),
 ('Hamburgers', 378),
 ('Asian', 371),
 ('Ice Cream, Gelato, Yogurt, Ices', 339),
 ('Indian', 332),
 ('Jewish/Kosher', 327),
 ('French', 319),
 ('Delicatessen', 294),
 ('Thai', 286)]